In [1]:
with open("./input.txt", "r") as file: 
    data = file.read().strip().split("\n")

# Part 1

In [2]:
import re

class FSItem: 
    """
    Filesystem item (folder, file)
    """
    def __init__(self, parent, name): 
        self.parent = parent
        self.name   = name
        
    @property
    def path(self):
        if self.parent is None: 
            return self.name
        return f"{self.parent.path}/{self.name}"
    
    @property
    def root(self):
        if self.parent is None: 
            return self
        return self.parent.root
    
    def __repr__(self):
        return f"{self.path} ({type(self).__name__}, {self.size})"

class File(FSItem): 
    def __init__(self, parent, name, size): 
        self.parent = parent
        self.name   = name
        self.size   = size

class Folder(FSItem): 
    def __init__(self, parent, name):
        self.parent = parent
        self.name   = name
        self.items  = {}
        
    def get(self, path):
        if path == "/":
            return self.root
        if path == "..": 
            return self.parent
        return self.items.setdefault(path, Folder(self, path))
    
    def ls(self):
        return [item.name for item in self.items.values()]
        
    def mkdir(self, name):
        self.items[name] = Folder(self, name)
        
    def touch(self, name, size):
        self.items[name] = File(self, name, size)
        
    def exists(self, name):
        return name in self.children
    
    def __len__(self):
        return len(self.items)
    
    @property
    def size(self):
        if len(self) == 0: 
            return 0
        
        return sum([item.size for item in self.items.values()])
    
    def find(self, callback):
        matches = []
        
        if callback(self): 
            matches.append(self)
            
        for item in self.items.values():
            if isinstance(item, Folder):
                matches.extend(item.find(callback))
            elif callback(item): 
                matches.append(item)
                
        return matches
            
        
class Shell: 
    def __init__(self):
        self.root = Folder(None, "/")
        self.cdir = self.root
        
    def execute(self, command): 
        if command.startswith("$ cd"): 
            self.cdir = self.cdir.get(path=command[5:])
    
class Program: 
    def __init__(self, shell=None):
        self.shell = shell or Shell()
        
    def parse(self, stdout):
        for line in stdout: 
            if line.startswith("$"): 
                self.shell.execute(line)
                continue
                
            search = re.match("(\d+) ([A-Za-z\.]+)", line)

            if search:
                size, filename = search.groups()
                self.shell.cdir.touch(filename, int(size))
        
        return self
    
def solve(data): 
    program = Program()
    program.parse(data)
    
    return sum(
        [
            folder.size 
            for folder 
            in program.shell.root.find(
                lambda i: isinstance(i, Folder) and i.size <= 100000
            )
        ]
    )

solve(data)

1723892

# Part 2

In [3]:
def solve(data): 
    program = Program()
    program.parse(data)
    
    total_disk_size  = 70000000
    total_used_space = program.shell.root.size 
    total_free_space = total_disk_size - total_used_space
    
    required_space = 30000000
    total_missing_space = max(0, required_space - total_free_space)
    
    # find all folders which are at least this big
    eligible_folders = program.shell.root.find(
        lambda i: isinstance(i, Folder) and i.size >= total_missing_space
    )
    
    # sort in order
    sorted_eligible_folders = sorted(eligible_folders, key=lambda f: f.size)
    
    # return the first (= smallest) folder which, if deleted, would allow to free
    # up the total_missing_space
    return sorted_eligible_folders[0].size

solve(data)

8474158